In [1]:
"""Hydrogen atom"""

__author__    = "Magnar Bjørgve"
__credit__    = ["Magnar Bjørgve"]

__date__      = "2021-02-16"

# Hydrogen atom


One of the simplest problems to solve, if not the simplest, is the Schrödinger equation for the Hydrogen atom. It's a
good starting point to get familiar with `vampyr`. Since, we know the analytic solution, so we have a solution we 
can refer to.

\begin{equation*}
\left[\hat{T} + \hat{V}\right]\phi = E\phi
\end{equation*}
Here $\hat{T}$ is the Kinetic energy operator
\begin{equation*}
\hat{T} = -\frac{1}{2}\nabla^2
\end{equation*}
and $\hat{V}$ is the potentail operator, which in this case only 
contain the nulear attraction
\begin{equation*}
\hat{V}(r) = \frac{-Z}{|R-r|}
\end{equation*}
where $Z=1$ and $R=(0,0,0)$ are the nuclear charge and position, respectively. We iterate the Schrödinger equation in its integral form
\begin{equation*}
\tilde{\phi}^{n+1} = -2 \hat{G}^{\mu^n}[\hat{V} \phi^n]
\end{equation*}
where $\hat{G}^\mu$ is the integral convolution operator with the bound-state Helmholtz kernel $G^{\mu} = e^{-\mu r}/r$ and $\mu^n = \sqrt{-2E^n}$ is a positive real number.


The analytic solution is know to be
\begin{align}
    \phi = e^{-|r|}
\end{align}
and energy $E = -0.5$

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np

def laplace_operator(D, f_tree):
    return D(D(f_tree, 0), 0) + D(D(f_tree, 1), 1) + D(D(f_tree, 2), 2)

def calculate_energy(phi_tree, V_tree):
    mra = phi_tree.MRA()
    d_oper = vp.ABGVDerivative(mra, 0.5, 0.5)
    return -0.5*vp.dot(laplace_operator(d_oper, phi_tree), phi_tree) + vp.dot(phi_tree, V_tree*phi_tree)

# Analytic nuclear potential: f_nuc(r) = Z/|r|
def f_nuc(r):
    Z = 1.0                 # Nuclear charge
    R = np.sqrt(r[0]**2 + r[1]**2 + r[2]**2)
    return -Z / R

# Analytic guess for wavefunction: f_phi(r) = exp(-r^2)
def f_phi(r):
    R2 = r[0]**2 + r[1]**2 + r[2]**2
    return np.exp(-R2)

# Analytic exact wavefunction for comparison
def f_phi_exact(r):
    return np.exp(-np.sqrt(r[0]**2 + r[1]**2 + r[2]**2))

# Set target precision
precision = 1.0e-3

# Define MRA basis and computational domain
k = 5                       # Polynomial order of MRA basis
world = [-20, 20]           # Computational domain [-L,L]^3 (a.u.)
MRA = vp.MultiResolutionAnalysis(order=k, box=world)

# Define projector onto the MRA basis
P_mra = vp.ScalingProjector(mra=MRA, prec=precision)

# Initialize the calculation
V = P_mra(f_nuc)               # Project analytic nuclear potential onto MRA
phi_n = P_mra(f_phi)           # Project analytic guess for wavefunction onto MRA
phi_n.normalize()              # Normalize the wavefunction guess
phi_exact = P_mra(f_phi_exact) # Project exact wavefunction onto MRA
phi_exact.normalize()          # Normalize the exact wavefunction
E_n = calculate_energy(phi_n, V)



In [ ]:
# Loop parameters
iteration = 0                  # Iteration counter
max_iter = 30                  # Maximum iterations 
thrs = precision # -1           # Convergence requirement. Set to -1 if you wish to limit using max_iter
update = 1.0                   # Initialize error measure (norm of wavefunction update)
# Minimization loop
while (update > thrs):
    if iteration > max_iter-1:
        break
    # Build Helmholtz operator from current energy
    mu = np.sqrt(-2*E_n)
    G = vp.HelmholtzOperator(mra=MRA, exp=mu, prec=precision)
    
    # Apply Helmholtz operator
    phi_np1 = -2*G(V*phi_n)
    
    # Compute wavefunction and energy update
    d_phi_n = phi_np1 - phi_n
    update = d_phi_n.norm()

    # Prepare for next iteration
    phi_n = phi_np1
    phi_n.normalize()
    E_n = calculate_energy(phi_n, V)

    # Collect output
    print(iteration, " |  E:", E_n, " |  d_phi:", update)
    iteration += 1